In [ ]:
from gurobipy import *
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from statistics import mean
import matplotlib.pyplot as plt

Read the data, the csv file can be obtained from https://www.pnas.org/doi/abs/10.1073/pnas.2016238117#supplementary-materials

In [ ]:
dat = pd.read_csv('pnas.2016238117.sd04.csv')

Choose the year of the dataset

In [ ]:
year = 2014
dat_sub = dat[dat['year']==year]
dat_sub = dat_sub.drop_duplicates()

Define variables for the problem using the data

In [ ]:
V = dat_sub.shape[0]
Q = dat_sub['crew_size']
Cq = np.log1p(dat_sub.iloc[:,20])
gamma = 0.25
k = (dat_sub['Prediction'] == 'Positive').sum() * gamma
num_samples = 1000
# define proportion of trafficking as a random variable
s_mean = dat_sub['.pred_1']
mar = .1
s=np.minimum(np.maximum(np.random.triangular(left=s_mean-mar, mode=s_mean, right=s_mean+mar, size=(num_samples, len(s_mean))),0),1)

S1 = [Q*(1-s[j]) for j in range(num_samples)]
S2 = [Q*s[j] for j in range(num_samples)]

In [ ]:
def error_rate(x):
    xn = x/x.sum()
    return xn

Choose the base TPR and TNR rates (con_base) along with the bias scenario (ca) and compute the TPR and TNR rate for each vessel

In [ ]:
con_base=0.5
ca = 'lpi'
ytpr, ytnr = [], []
con_set = dat_sub[['fishing_hours', 'average_voyage_duration_hours', 'average_loitering_duration_hours']]
for h in range(V):
    if ca == 'hpi':
        ccon = error_rate(con_set.iloc[h,:])
        if(ccon.min() < con_base and ccon.max() < con_base):
            ytpr.append(con_base)
            ytnr.append(con_base)
        if(ccon.min() < con_base and ccon.max() > con_base):
            ytpr.append(ccon.max())
            ytnr.append(con_base)
        if(ccon.min() > con_base and ccon.max() > con_base):
            ytpr.append(ccon.max())
            ytnr.append(ccon.min())
    if ca == 'hni':
        ccon = error_rate(con_set.iloc[h,:])
        if(ccon.min() < con_base and ccon.max() < con_base):
            ytpr.append(con_base)
            ytnr.append(con_base)
        if(ccon.min() < con_base and ccon.max() > con_base):
            ytpr.append(con_base)
            ytnr.append(ccon.max())
        if(ccon.min() > con_base and ccon.max() > con_base):
            ytpr.append(ccon.min())
            ytnr.append(ccon.max())
    if ca == 'hpln':
        ccon = error_rate(con_set.iloc[h,:])
        if(ccon.min() == 0):
            ytpr.append(ccon.max())
            ytnr.append(ccon.min() + float(np.random.uniform(0,.1,1)))
        else:
            ytpr.append(ccon.max())
            ytnr.append(ccon.min())
    if ca == 'lpi':
        ccon = error_rate(con_set.iloc[h,:])
        if(ccon.min() < con_base and ccon.max() < con_base):
            ytpr.append(con_base)
            ytnr.append(ccon.min())
        if(ccon.min() < con_base and ccon.max() > con_base):
            ytpr.append(con_base)
            ytnr.append(ccon.min())
        if(ccon.min() > con_base and ccon.max() > con_base):
            ytpr.append(con_base)
            ytnr.append(con_base)
    if ca == 'lni':
        ccon = error_rate(con_set.iloc[h,:])
        if(ccon.min() < con_base and ccon.max() < con_base):
            ytpr.append(ccon.min())
            ytnr.append(con_base)
        if(ccon.min() < con_base and ccon.max() > con_base):
            ytpr.append(ccon.min())
            ytnr.append(con_base)
        if(ccon.min() > con_base and ccon.max() > con_base):
            ytpr.append(con_base)
            ytnr.append(con_base)
    if ca == 'hnlp':
        ccon = error_rate(con_set.iloc[h,:])
        if(ccon.min() == 0):
            ytnr.append(ccon.max())
            ytpr.append(ccon.min() + float(np.random.uniform(0,.1,1)))
        else:
            ytnr.append(ccon.max())
            ytpr.append(ccon.min())

Compute coefficients for the optimization problem

In [ ]:
ytpr, ytnr = np.array(ytpr), np.array(ytnr)
coeff2 = [[a*b for a,b in zip(S2[j],ytpr)] for j in range(num_samples)]
coeff3 = [[a*b for a,b in zip(S1[j],ytnr)] for j in range(num_samples)]
coq = [S1[j]*(1-ytnr)+S2[j]*(1-ytpr) for j in range(num_samples)]

Fix RHS for the opimization problem

In [ ]:
B = 10000
K = k
L = 10000

Formulate DTRAP-BIN for GFW

In [ ]:
m = Model()
m.ModelSense = 1  # minimize
# Add variables
x = m.addVars(V, vtype=GRB.BINARY, name='x')
error = m.addVars(num_samples, obj=1.0/num_samples, name='error')
# Set constraints
m.addConstr(quicksum(Cq.iloc[i]*x[i] for i in range(V)) <= B, name="resources")
m.addConstrs((quicksum(coeff2[j][i]*x[i] for i in range(V)) >= K for j in range(num_samples)), name="HT")
m.addConstrs((quicksum(coeff3[j][i]*x[i] for i in range(V)) <= L for j in range(num_samples)), name="NHT")
m.addConstrs((error[j] == quicksum(coq[j].iloc[i]*x[i] for i in range(V)) for j in range(num_samples)), name='error')
m.update()

For HNI bias scenario, change the default optimality gap

In [ ]:
if ca == 'hni':
    m.Params.MIPGap = .005

Solve the optimization problem

In [ ]:
m.optimize()

Compute values for the optimal allocation. These are used for creating Table 5 in the manuscript.

In [ ]:
num_monitor = sum([v.X for k,v in x.items()])
obj_vals = [v.X for k,v in error.items()]
obj = np.mean(obj_vals)
k_vals = [quicksum(coeff2[j][i]*x[i] for i in range(V)).getValue() for j in range(num_samples)]
l_vals = [quicksum(coeff3[j][i]*x[i] for i in range(V)).getValue() for j in range(num_samples)]
sol = [v.X for k,v in x.items()]

Save/load pickle object

In [ ]:
with open(f'sol_{ca}_{year}.pkl', 'wb') as f:
    pickle.dump([num_monitor, obj_vals, k_vals, l_vals, sol], f)
# with open(f'sol_{ca}_{year}.pkl', 'rb') as f:
#     num_monitor, obj_vals, k_vals, l_vals, sol = pickle.load(f)

In [ ]:
print(num_monitor)
print(round(mean(obj_vals), 2))
print(round(sum([Cq.iloc[i]*sol[i] for i in range(V)]), 2))
print(round(mean(k_vals), 2))
print(round(mean(l_vals), 2))

Get the allocations using the ML model

In [ ]:
alloc_ML = np.zeros(V)
alloc_ML[np.argsort(s_mean)[-int(num_monitor):]] = 1

In [ ]:
print(round(mean(quicksum(coq[j]*alloc_ML).getValue() for j in range(num_samples)), 2))
print(round(sum(Cq.iloc[i]*alloc_ML[i] for i in range(V)), 2))
print(round(mean([quicksum(coeff2[j][i]*alloc_ML[i] for i in range(V)).getValue() for j in range(num_samples)]), 2))
print(round(mean([quicksum(coeff3[j][i]*alloc_ML[i] for i in range(V)).getValue() for j in range(num_samples)]), 2))

Plotting errors (Figure 3)

In [ ]:
obj = np.mean(obj_vals)
plt.rcParams['font.size'] = '12'
plt.hist((obj_vals-obj)*100/obj, bins=30, alpha=0.75, label='DTRAP', density=True, color='#fde725')
plt.hist(([quicksum(coq[j]*alloc_ML).getValue() for j in range(num_samples)]-obj)*100/obj, bins=30, alpha=0.75, label='ML model', density=True, color='#440154')
plt.legend(loc='upper right')
plt.xlabel('Errors', fontsize=14)
plt.ylabel('Probability', fontsize=14)
plt.savefig(f'{ca}_errors.pdf', bbox_inches='tight', transparent=True)

Plotting trafficking cases detected (Figure 3)

In [ ]:
plt.rcParams['font.size'] = '12'
plt.hist((k_vals-np.mean(k_vals))*100/np.mean(k_vals), bins=30, alpha=0.75, label='DTRAP', density=True, color='#fde725')
plt.hist(([quicksum(coeff2[j][i]*alloc_ML[i] for i in range(V)).getValue() for j in range(num_samples)]-np.mean(k_vals))*100/np.mean(k_vals), bins=30, alpha=0.75, label='ML model', density=True, color='#440154')
plt.legend(loc='upper right')
plt.xlabel('Trafficking', fontsize=14)
plt.ylabel('Probability', fontsize=14)
plt.savefig(f'{ca}_K.pdf', bbox_inches='tight', transparent=True)

Plotting non-trafficking cases monitored (Figure 3)

In [ ]:
plt.rcParams['font.size'] = '12'
plt.hist((l_vals-np.mean(l_vals))*100/np.mean(l_vals), bins=30, alpha=0.75, label='DTRAP', density=True, color='#fde725')
plt.hist(([quicksum(coeff3[j][i]*alloc_ML[i] for i in range(V)).getValue() for j in range(num_samples)]-np.mean(l_vals))*100/np.mean(l_vals), bins=30, alpha=0.75, label='ML Model', density=True, color='#440154')
plt.legend(loc='upper right')
plt.xlabel('Non-trafficking', fontsize=14)
plt.ylabel('Probability', fontsize=14)
plt.savefig(f'{ca}_L.pdf', bbox_inches='tight', transparent=True)